# [CMI-SleepState-Detection](https://www.kaggle.com/competitions/child-mind-institute-detect-sleep-states/overview)
## Child Mind Institute - Detect Sleep States
### Detect sleep onset and wake from wrist-worn accelerometer data
_______________________________________________________________________ 
# Author Details:
- Name: Najeeb Haider Zaidi
- Email: zaidi.nh@gmail.com
- Profiles: [Github](https://github.com/snajeebz)  [LinkedIn](https://www.linkedin.com/in/najeebz) [Kaggle](https://www.kaggle.com/najeebz)
- License: Private, Unlicensed, All the files in this repository under any branch are Prohibited to be used commercially or for personally, communally or privately unless permitted by author in writing.
- Copyrights 2023-2024 (c) are reserved only by the author: Najeeb Haider Zaidi
________________________________________________________________________
# Attributions:
The Dataset has been provided by Child Mind Institute. in [Kaggle Competition](https://www.kaggle.com/competitions/child-mind-institute-detect-sleep-states/overview) which the author is participating in and authorized to use the dataset solely for the competition purposes.
________________________________________________________________________

In [1]:
!pip install pandarallel


  Preparing metadata (setup.py) ... done
  Created wheel for pandarallel: filename=pandarallel-1.6.5-py3-none-any.whl size=16672 sha256=e56dc767ee9d5ce83f7fce7f5b8d8cc587847422c8bfbe415bdf8fbc00f89f57
  Stored in directory: /root/.cache/pip/wheels/50/4f/1e/34e057bb868842209f1623f195b74fd7eda229308a7352d47f
Successfully built pandarallel


In [2]:
import numpy as np # linear algebra
import pandas as pd# data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from pandarallel import pandarallel
import plotly.express as px
import matplotlib.pyplot as plt
from datetime import datetime as dts
pd.set_option('display.max_row', 500)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_colwidth', None)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
from tqdm import tqdm
tqdm.pandas()
pandarallel.initialize(progress_bar=True)

/kaggle/input/child-mind-institute-detect-sleep-states/train_series.parquet
/kaggle/input/child-mind-institute-detect-sleep-states/sample_submission.csv
/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv
/kaggle/input/child-mind-institute-detect-sleep-states/test_series.parquet
/kaggle/input/later-data/Full_merged.pkl
/kaggle/input/later-data/full_clustered .pkl
/kaggle/input/later-data/cluster_model.joblib
INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [22]:
events=pd.read_csv('/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv')

In [27]:
df_full=pd.read_pickle('/kaggle/input/later-data/full_clustered .pkl')

# Analyzing the relationshiip between sleep and cluster

In [ ]:
df_sleep=df.loc[df['sleep']==1]
df_onset=df.loc[df['sleep']==0]

In [ ]:
df_sleep['cluster'].mode()

In [ ]:
df_onset['cluster'].mode()

In [ ]:
#df['timestamp']=df['timestamp'].parallel_apply(lambda x: pd.to_datetime(x,utc=True))

In [ ]:
df.info()

In [ ]:
df.to_pickel('cluster_timestamped.pkl')

In [ ]:
events

In [23]:
events['step'].isnull().sum()

4923

In [25]:
event_df=events.dropna()

In [ ]:
event_df

In [26]:
df['window']=np.nan

In [28]:
ind=df.index[df['event'].isna()==False]
print(ind)

Index([     4992,     10932,     20244,     27492,     39996,     44400,
           57240,     62856,     91296,     97860,
       ...
       127859076, 127865244, 127876812, 127883064, 127892916, 127901112,
       127910520, 127914564, 127928580, 127935564],
      dtype='int64', length=9585)


In [29]:
# 30 min window on change event, with single timestep:5sec. 
c=0
for i in tqdm(ind):
    a=i-360
    b=i+360
    df['window'].loc[a:i]=int(c)
    c=c+1
    df['window'].loc[i:b]=int(c)
    c=c+1
print('Windows Created: ', c)    



100%|██████████| 9585/9585 [00:08<00:00, 1139.92it/s]

Windows Created:  19170


In [30]:
df=df[df['window'].isna()==False]

In [31]:
df['window'].isnull().sum()

0

In [3]:
#df.to_pickle('ts_win_60_diff.pkl')
df=pd.read_pickle('ts_win_60_diff.pkl')

In [40]:
df.groupby(df['sleep']).describe().transpose()

sleep                       0.0           0.2
step        count  3.456474e+06  3.454166e+06
            mean   2.141468e+05  2.143393e+05
            std    1.413030e+05  1.412488e+05
            min    0.000000e+00  9.360000e+02
            25%    9.526225e+04  9.534000e+04
            50%    2.004330e+05  2.004030e+05
            75%    3.170390e+05  3.176540e+05
            max    7.397520e+05  7.393910e+05
anglez      count  3.456474e+06  3.454166e+06
            mean  -9.595487e+00 -9.819754e+00
            std    3.897369e+01  4.760664e+01
            min   -9.000000e+01 -9.000000e+01
            25%   -3.938540e+01 -5.052565e+01
            50%   -1.123030e+01 -1.626280e+01
            75%    1.693547e+01  3.097840e+01
            max    8.998630e+01  8.997170e+01
enmo        count  3.456474e+06  3.454166e+06
            mean   3.775361e-02  7.744349e-03
            std    7.064855e-02  2.023900e-02
            min    0.000000e+00  0.000000e+00
            25%    1.500000e-03  0.000000e+00
            50%    1.710000e-02  0.000000e+00
            75%    4.300000e-02  9.600000e-03
            max    5.507400e+00  3.458200e+00
cluster     count  3.456474e+06  3.454166e+06
            mean   2.415565e-01  2.590511e-01
            std    1.971435e-01  2.216293e-01
            min    0.000000e+00  0.000000e+00
            25%    1.666667e-01  0.000000e+00
            50%    1.666667e-01  1.666667e-01
            75%    5.000000e-01  5.000000e-01
            max    5.000000e-01  5.000000e-01
window      count  3.456474e+06  3.454166e+06
            mean   9.586043e+03  9.582677e+03
            std    5.534488e+03  5.533265e+03
            min    0.000000e+00  1.000000e+00
            25%    4.795000e+03  4.790000e+03
            50%    9.585500e+03  9.583000e+03
            75%    1.438025e+04  1.437500e+04
            max    1.916900e+04  1.916800e+04
sd_anglez_1 count  3.456463e+06  3.454166e+06
            mean   1.463170e+01  2.445324e+00
            std    1.265082e+01  7.940958e+00
            min    0.000000e+00  0.000000e+00
            25%    3.231413e+00  0.000000e+00
            50%    1.297995e+01  3.836010e-02
            75%    2.242456e+01  1.655767e-01
            max    8.946939e+01  8.933976e+01
sd_enmo_1   count  3.456463e+06  3.454166e+06
            mean   3.001441e-02  2.779273e-03
            std    4.502104e-02  1.144730e-02
            min    0.000000e+00  0.000000e+00
            25%    4.221473e-03  0.000000e+00
            50%    1.714969e-02  6.513392e-05
            75%    4.002313e-02  3.165870e-04
            max    2.291263e+00  1.071057e+00

In [33]:
df['timestamp']=df['timestamp'].parallel_apply(lambda x: pd.to_datetime(x,utc=True))

# Normalizing cluster and sleep

In [4]:
df_n=df


In [ ]:
df_n['cluster']=df_n['cluster']/6
df_n['sleep']=df_n['sleep']/5
df_n.describe()

## Analyzing the window

# Adding rolling Deviation

In [7]:
df_n['sd_anglez_1']=df_n['anglez'].rolling(window=12).std()
df_n['sd_enmo_1']=df_n['enmo'].rolling(window=12).std()
df_n.describe()


,step,anglez,enmo,sleep,cluster,window,sd_anglez_1,sd_enmo_1
count,6.910640e+06,6.910640e+06,6.910640e+06,6.910640e+06,6.910640e+06,6.910640e+06,6.910629e+06,6.910629e+06
mean,2.142430e+05,-9.707589e+00,2.275398e-02,1.999332e-02,4.171681e-02,9.584361e+03,8.540537e+00,1.640137e-02
std,1.412759e+05,4.350354e+01,5.409548e-02,2.000000e-02,3.498704e-02,5.533877e+03,1.219402e+01,3.556291e-02
min,0.000000e+00,-9.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,9.530800e+04,-4.534660e+01,0.000000e+00,0.000000e+00,0.000000e+00,4.792000e+03,2.289136e-02,0.000000e+00
50%,2.004120e+05,-1.312360e+01,4.500000e-03,0.000000e+00,2.777778e-02,9.584000e+03,9.267053e-01,1.113111e-03
75%,3.173210e+05,2.323820e+01,2.480000e-02,4.000000e-02,8.333333e-02,1.437700e+04,1.512493e+01,1.981920e-02
max,7.397520e+05,8.998630e+01,5.507400e+00,4.000000e-02,8.333333e-02,1.916900e+04,8.946939e+01,2.291263e+00


In [50]:
for i in range(10):
        title="Sleep: "+ str(df_n['sleep'][df_n['window']==i].mean())
        fig=px.line(y=df_n['sleep'][df_n['window']==i],x=df_n['timestamp'][df_n['window']==i])
        fig.add_scatter(y=df_n['cluster'][df_n['window']==i],x=df_n['timestamp'][df_n['window']==i], name='cluster')
        fig.add_scatter(y=df_n['enmo'][df_n['window']==i],x=df_n['timestamp'][df_n['window']==i], name='sleep')
        fig.update_layout(title=title)
        fig.show()

In [20]:
df.resample('60min', label='right').first()


TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Index'

In [ ]:
fig=px.line(y=df['enmo'][df['window']==4],x=df['timestamp'][df['window']==4])
fig.add_scatter(y=df['cluster'][df['window']==4],x=df['timestamp'][df['window']==4])
fig.show()

In [ ]:
fig=px.line(y=df['enmo'][df['window']==2],x=df['timestamp'][df['window']==2])
fig.add_scatter(y=df['cluster'][df['window']==2],x=df['timestamp'][df['window']==2])
fig.show()

In [ ]:
fig=px.line(y=df['enmo'][df['window']==1],x=df['timestamp'][df['window']==1])
fig.add_scatter(y=df['cluster'][df['window']==1],x=df['timestamp'][df['window']==1])
fig.show()

In [5]:
df_n.columns
df_n['sd_enmo_1']=pd.to_numeric(df_n['sd_enmo_1'])

df_n['sd_enmo_1'].fillna('0.0',inplace=True)
df_n['sd_enmo_1'].isnull().sum()

0

In [6]:
df_n['sd_anglez_1']=pd.to_numeric(df_n['sd_anglez_1'])
df_n['sd_anglez_1'].fillna('0.0',inplace=True)
df_n['sd_anglez_1'].isnull().sum()

0

In [7]:
df_n['sleep'].unique()

array([0.  , 0.04])

In [8]:
df['sleep'].unique()

array([0.  , 0.04])

In [24]:
df['sleep_bool']=df['sleep'].replace({0.04: True, 0: False})

array([False,  True])

In [17]:
df['sleep_bool'].value_counts()

sleep_bool
False    3456474
True     3454166
Name: count, dtype: int64

In [10]:
from sklearn.model_selection import train_test_split
X=df_n[['sd_anglez_1','sd_enmo_1','anglez','enmo','cluster']]
y=df[['sleep_bool']]
X_train, X_test, y_train, y_test =train_test_split(X,y,test_size=0.2)

In [11]:
from joblib import dump, load
model=clf = load('rf_model.joblib')

In [29]:
from sklearn.ensemble import RandomForestClassifier
rf =RandomForestClassifier(n_jobs=-1,verbose=1) 
print ('Training the model')
rf.fit(X_train,y_train)
print ('Saving the model')
from joblib import dump, load
dump(rf, 'rf_model.joblib')

Training the model


[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed: 20.7min
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed: 44.9min finished


Saving the model


['rf_model.joblib']

In [18]:
from sklearn.metrics import accuracy_score
y_pred=model.predict(X_test)
#cm=confusion_matrix(y_test, y_pred)
#sns.heatmap(cm, annot=True)
#print("Confusion Matrix: ", cm)
#print("Precision Score : ", precision_score(y_test,y_pred)) #precision measures the proportion of true positive predictions among all positive instances. how many of survived predicted actually survived, doesn't verifies 0's 70 survived as preicted whereas actually 92 survived so 70/92 will be the precision.  if we predicted 70 survived, so presion will tell how many of those 70 predicted survived matches the actual row by row data. It checkes all positives and verifies if the answer is true for each row?
#print("Recall Score: ", recall_score(y_test,y_pred, average='macro')) #Recall measures the proportion of true positive predictions among all actual positive instalnces. If we predicted 100 survived correctly whereas actually 100 survived out of which 67 predicted correctly so recall will be 0.67
#print("F1 Score: ",f1_score(y_test,y_pred)) 
print("Accuracy: ",accuracy_score(y_test,y_pred)) 

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:   28.7s


Accuracy:  0.9753083650718313


[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:  1.0min finished


In [24]:
pred=y_test

In [26]:
pred['predicted']=y_pred

In [29]:
def prepare(df):
    df['sd_anglez_1']=pd.to_numeric(df_n['anglez'].rolling(window=12).std())
    df['sd_enmo_1']=pd.to_numeric(df_n['enmo'].rolling(window=12).std())
    df['sd_anglez_1'].fillna('0.0',inplace=True)
    df['sd_enmo_1'].fillna('0.0',inplace=True)
    return(df)
    